<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/Attack_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attack

# Libraries

In [1]:
# Importing necesary libraries
# Libraries for correct code execution 

import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import random
import csv
import re
import array
import seaborn as sns
from tqdm import tqdm

from functools import reduce
from random import random, gauss
from math import modf, pi, cos, sin, sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from sklearn.mixture import GaussianMixture
import os, time, math, csv, joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import os, time, math, csv, joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

sns.set_theme()

# Load Enviroment

In [2]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [3]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Change the number to change the paths.
worker = workers[0]

if worker == "Ronald":
  path = "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

# Pre-procesing

In [4]:
input_path = path + "/Spectrum/NN/new_data_working_df.csv"

working_df = pd.read_csv(input_path)
working_df['Distance_km'] = working_df['Distance_km'].astype(int)

# Utils

## Generate Train/Test data

In [5]:
def create_df(input_data_path, distances, nsymbols = 2048, min_dist = 0, max_dist = 3000):
  X=None
  Y=[]
  colnames=['i'+str(i) for i in range(nsymbols)]

  for d in distances:
    dist=d*span_length
    if dist<min_dist or dist>max_dist: continue
    filename='consts_'+str(d)+'span.csv'
    df_aux=pd.read_csv(input_data_path+'/'+filename, sep=",", header=None)
    df_aux = df_aux.T
    df_aux.columns=colnames
    Y=Y+[dist]*df_aux.shape[0]
    if X is None: X=df_aux
    else: X=X.append(df_aux)
  return X, Y

In [6]:
def create_df_v2(working_df, distances, nsymbols = 2048, span_length=80, min_dist = 0, max_dist = 3000):
  X=None
  Y=np.array([])

  for d in distances:
    distance_working_df = working_df.loc[working_df['Distance_km'] == d]
    
    d_df = distance_working_df.iloc[:, 3].to_numpy()
    data_df = distance_working_df.iloc[:, 6:distance_working_df.shape[1]]
    Y = np.append(Y, d_df)

    if X is None: X=data_df
    else: X=X.append(data_df)
  Y = Y.astype('int').tolist()

  return X, Y

In [7]:
#input_data_path=path+"/rawData"
#distances=[i for i in range(1,26)]
distances = [x*80 for x in range(1, 26)]
X, Y = create_df_v2(working_df, distances, nsymbols = 2048, span_length=80,  min_dist = 0, max_dist = 3000)

## Get indexes for train/test split

In [8]:
def get_idx_train_test_split(working_df, distances, trainingProp = 0.8):

  rows_training = np.array([])
  rows_testing = np.array([])
  for d in distances:  
    distance_working_df = working_df.loc[working_df['Distance_km'] == d]

    rows_mixed=np.random.permutation(distance_working_df.shape[0])

    training_amt = math.ceil(distance_working_df.shape[0]*trainingProp)
    testing_amt = distance_working_df.shape[0] - training_amt

    rows_training = np.append(rows_training, rows_mixed[:training_amt])
    rows_testing = np.append(rows_testing, rows_mixed[-testing_amt:])
  
  rows_training = rows_training.astype('int').tolist()
  rows_testing = rows_testing.astype('int').tolist()

  return rows_training, rows_testing

In [9]:
rows_training, rows_testing = get_idx_train_test_split(working_df, distances, trainingProp = 0.8)

In [10]:
def test_train_indexes_v2():
  train_idxs2 = []
  test_idxs2 = []
  for i in range(25):
      for j in range(25):
          test_idxs2.append(50*(i+1)-1-j)

  for i in range(25):
    for j in range(2):
      train_idxs2.append(50*i + j)
  return train_idxs2, test_idxs2

# GD

## Gradient Descent Function

In [11]:
def grad_b (M,m_og,m_tg,b,beta):
  return 2*beta*(M @ m_og + b - m_tg)

In [12]:
def grad_M(M,cov_og,cov_tg,alpha):
    error = M @ cov_og @ np.transpose(M) - cov_tg  

    m1 = (2*error[0][0] * (2*cov_og[0][0]*M[0][0] + 2*cov_og[0][1]*M[0][1]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][0]*M[1][0] + cov_og[0][1]*M[1][1]))
    
    m2 = (2*error[0][0] * (2*cov_og[1][1]*M[0][1] + 2*cov_og[0][1]*M[0][0]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][1]*M[1][0] + cov_og[1][1]*M[1][1]))
    
    m3 = (2*error[1][1] * (2*cov_og[0][0]*M[1][0] + 2*cov_og[0][1]*M[1][1]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][0]*M[0][0] + cov_og[0][1]*M[0][1]))

    m4 = (2*error[1][1] * (2*cov_og[0][0]*M[1][1] + 2*cov_og[0][1]*M[1][0]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][1]*M[0][0] + cov_og[1][1]*M[0][1]))
    
    return alpha*np.array([[m1, m2], [m3, m4]])

In [13]:
def gradient_descent(alpha,beta,m_tg,m_og,cov_tg,cov_og,nu,log):
  Ms = []
  bs = []
  M = np.random.rand(2,2)
  #M = np.array([[1, 0], [0, 1]])
  b = np.random.rand(2,1)
  #b = np.array([[0], [0]])
  #for i in range(100000):
  i = 0
  while True:
    # print('From:', M @ cov_source @ np.transpose(M))
    # print('To:', cov_target)
    L = loss(alpha,beta,m_tg,m_og,cov_tg,cov_og,M,b) 
    if  L < 1e-20:
        Ms.append(M)
        bs.append(b)
        break
    b = b - nu*grad_b(M,m_og,m_tg,b,beta)
    M = M - nu*grad_M(M,cov_og,cov_tg,alpha)
    i+= 1
    if (i>= 5000 and i <= 5025) or (i>= 5975 and i <= 6000):
      Ms.append(M)
      bs.append(b)
    if not i%5000 and log: print(L)
  if log: print("-"*25)
  return Ms,bs

## Compute parameters and mean/covariance

In [14]:
def compute_mean_and_cov(data):
  aux_x = [] # Reales
  aux_y = [] # Imag
  for obs in data:
    aux_x.append(obs[0])
    aux_y.append(obs[1])
  return np.array([[np.mean(aux_x)],[np.mean(aux_y)]]), np.cov(aux_x,aux_y)


def total_loss(m_mod,m_tg,cov_mod,cov_tg):
  return (sum(sum(np.power(cov_mod-cov_tg,2))) + sum(np.power(m_mod - m_tg,2)))[0]


def loss(alpha,beta,m_tg,m_og,cov_tg,cov_og,M,b):
  first = alpha*sum(sum(np.power(M @ cov_og @ np.transpose(M) - cov_tg, 2)))
  second = beta*sum(np.power(M @ m_og + b - m_tg, 2))
  a = first+second
  return a

In [ ]:
def compute_parameters (const,method,source,target, params = None, log = False):
  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  q_target = []

  for i in range(len(source)):

    q_source += [[x.real,x.imag] for x in source.values.tolist()[i] if x.real < limits[0][0] and x.real > limits[0][1] and x.imag < limits[1][0] and x.imag > limits[1][1]]
    q_target += [[x.real,x.imag] for x in target.values.tolist()[i]if x.real < limits[0][0] and x.real > limits[0][1] and x.imag < limits[1][0] and x.imag > limits[1][1]]

  unbiased = (len(q_source)/(len(q_source)-1) * 125/126)

  mean_source,cov_source = compute_mean_and_cov(q_source)
  mean_target, cov_target= compute_mean_and_cov(q_target)

  cov_source *= unbiased
  cov_target *= unbiased

  if method == "GD":
    if (params is None):
      alpha = 3/4
      beta = 1/4
    else:
      alpha = params["alpha"] if params["alpha"]>0 and params["alpha"]<=1 and params["alpha"]>params["beta"]  else 3/4
      beta = params["beta"] if params["beta"]>=0 and params["beta"]<1 and params["beta"]<params["alpha"]  else 1/4

    M, b = gradient_descent(alpha,beta,mean_target,mean_source,cov_target,cov_source, 0.5,log)
    return M,b

  elif method == "Z":
    return mean_source,cov_source,mean_target,cov_target

## Modify GD/Z

In [ ]:
def modify_const_GD(const,source,M,b, target = None, return_plots=False):

  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  indexes = []

  for i in range(len(source)):
    row_source = source.values.tolist()[i]
    for j in range(len(row_source)):
      point_source = row_source[j]
      if point_source.real < limits[0][0] and point_source.real > limits[0][1] and point_source.imag < limits[1][0] and point_source.imag > limits[1][1]:
        q_source += [[point_source.real,point_source.imag]]
        indexes.append([i,j])

  if target is not None:
    q_target = []

    for i in range(len(source)):
      row_target = target.values.tolist()[i]
      for j in range(len(row_source)):
        point_target = row_target[j]
        if point_target.real < limits[0][0] and point_target.real > limits[0][1] and point_target.imag < limits[1][0] and point_target.imag > limits[1][1]:
          q_target += [[point_target.real,point_target.imag]]
      
  mults = [np.array([[x[0]],[x[1]]]) for x in q_source]
  new_points = []

  for mult in mults:
    res = (M @ mult + b).tolist()

    # --------- Old line ---------
    #new_points.append([res[0][0],res[1][0]])
    nested_check = any(isinstance(i, list) for i in res[0])
    if nested_check:
      new_points.append([res[0][0][0],res[0][1][0]])
    else:
      new_points.append([res[0][0],res[1][0]])

  if target is not None:
    if return_plots:
      generate_comparison_plot(new_points,q_target)
      return new_points, q_target

  return new_points, indexes

In [ ]:
def modify_const_Z(const,source,mean_source,cov_source,mean_target,cov_target,target = None, return_plots=False):
  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  indexes = []

  for i in range(len(source)):
    row_source = source.values.tolist()[i]
    for j in range(len(row_source)):
      point_source = row_source[j]
      if point_source.real < limits[0][0] and point_source.real > limits[0][1] and point_source.imag < limits[1][0] and point_source.imag > limits[1][1]:
        q_source += [[point_source.real,point_source.imag]]
        indexes.append([i,j])

  if target is not None:
    q_target = []

    for i in range(len(source)):
      row_target = target.values.tolist()[i]
      for j in range(len(row_source)):
        point_target = row_target[j]
        if point_target.real < limits[0][0] and point_target.real > limits[0][1] and point_target.imag < limits[1][0] and point_target.imag > limits[1][1]:
          q_target += [[point_target.real,point_target.imag]]

  mults = [np.array([[x[0]],[x[1]]]) for x in q_source]
  new_points = []

  w, v = np.linalg.eig(cov_source)
  S1 = v @ np.diag(np.sqrt(w)) @ np.linalg.inv(v)
  w, v = np.linalg.eig(cov_target)
  S2 = v @ np.diag(np.sqrt(w)) @ np.linalg.inv(v)

  for mult in mults:
    normalized = np.linalg.inv(S1) @ (mult - mean_source)
    denormalized = S2 @ normalized + mean_target

    new_points.append([denormalized[0][0], denormalized[1][0]])

  if target is not None:
    if return_plots:
      generate_comparison_plot(new_points,q_target)
      

  return new_points, indexes

In [ ]:
def L2dist(a,b):
    return math.sqrt(math.pow(a[0]-b[0],2)+math.pow(a[1]-b[1],2))

# Main

In [ ]:
def symbol2symbol_main(method, quadrants, input_data_path, distances, output_path, params = None, nsymbols = 2048, min_dist = 0, max_dist = 3000):
  
  if (params is None):
    output_features = "/{}_alpha_75_beta_25_features.csv".format(str(method))
  else:
    decimal_alpha = modf(params['alpha'])
    decimal_alpha  = round(decimal_alpha[0], 2)
    decimal_alpha = re.findall('..(.*)', str(decimal_alpha))[0]

    decimal_beta = modf(params['beta'])
    decimal_beta  = round(decimal_beta[0], 2)
    decimal_beta = re.findall('..(.*)', str(decimal_beta))[0]

    output_features = "/{}_alpha_{}_beta{}_features.csv".format(str(method), str(decimal_alpha), str(decimal_beta))

  # Get input data
  X, Y = create_df(input_data_path, distances, nsymbols, min_dist, max_dist)

  X = X.applymap(strToTuple)

 
  train_idxs2, test_idxs2 = test_train_indexes_v2()

  X_train2 = X.iloc[train_idxs2].reset_index(drop = "True")
  X_test2 = X.iloc[test_idxs2].reset_index(drop = "True") 

  data_new = {}

  for i in range(0,25):
    data_new[str(80*(i+1))] = {"Train":X_train2.iloc[i*2:(i*2+2)].reset_index(drop = "True"),"Test":X_test2.iloc[i*25:(i*25+25)].reset_index(drop = "True")}  
  display(data_new)
  
  first = True
  modify_all_data_df = pd.DataFrame()
  for i in tqdm(range(25)):
      print('\n')
      print("i = ",i)
      modify_data_df = pd.DataFrame()
      for j in range(i+1,25):
          print(" j = ",j)
          source = data_new[str(distances[i]*80)]['Train']
          target = data_new[str(distances[j]*80)]['Train']
          source_test = data_new[str(distances[i]*80)]['Test']

          source_test2 = source_test.copy()

          for quadrant in quadrants:
              print("   quadrant = ", quadrant)
              if method == "GD":
                M, b = compute_parameters(quadrant, 'GD', source, target, params=params)
                new_points, indexes = modify_const_GD(quadrant, source_test, M, b)
              elif method == "Z":
                mean_source,cov_source,mean_target,cov_target = compute_parameters(quadrant, 'Z', source, target, params=None)
                new_points, indexes = modify_const_Z(quadrant,source_test,mean_source,cov_source,mean_target,cov_target)

              for k in range(len(indexes)):
                  source_test2.iloc[indexes[k][0],indexes[k][1]] = complex(new_points[k][0], new_points[k][1])

          F=[]
          for k in range(source_test2.shape[0]):
              data2=list(source_test2.iloc[k,:])
              data2=[[float(d.real), float(d.imag)] for d in data2]
              gmm = GaussianMixture(n_components=16, random_state=0, means_init=my_centers).fit(data2)
              mus=gmm.means_
              sigmas=gmm.covariances_

              features=[distances[i]*80, distances[j]*80]

              for z in selCP_pos:
                  mindist=None
                  k_inc=None
                  for w in range(16):
                      d=L2dist(mus[w],z)
                      if mindist is None or mindist>d:
                          mindist=d
                          k_inc=w

                  covmat=np.concatenate(list(sigmas[k_inc])).ravel().tolist()
                  features = [*features, *mus[k_inc], *covmat]
              F.append(features)
          
          header=['original_dist', 'target_dist']
          
          for j in selCP:
              header=[*header,*['mu_r_'+str(j),'mu_i_'+str(j),'sigma_rr_'+str(j),'sigma_ri_'+str(j),'sigma_ir_'+str(j),'sigma_ii_'+str(j)]]
   
          with open(output_path + output_features, 'a', encoding='UTF8', newline='') as f:
              writer = csv.writer(f)
              # write the header
              if first:
                writer.writerow(header)
                first = False
              # write multiple rows
              writer.writerows(F)   

## Uitls Main

In [ ]:
def strToTuple(s):
    s_aux=s.split("i")
    s=s_aux[0]+"j"
    return complex(s)

def strToTuple_v2(s):
    return complex(s)

In [ ]:
def test_train_indexes_v2():
  train_idxs2 = []
  test_idxs2 = []
  for i in range(25):
      for j in range(25):
          test_idxs2.append(50*(i+1)-1-j)

  for i in range(25):
    for j in range(2):
      train_idxs2.append(50*i + j)
  return train_idxs2, test_idxs2

In [ ]:
# PLEASE SELCET THE METHOD AND QUADRANTS THAT ARE DESIRED TO BE TESTED
method = "GD" # or method = "GD"
quadrants = [1, 7, 10, 15]

#Output mod files
output_path = path + '/modifiedData/'+ method

# PLEASE review that the path below is correct
input_data_path=path+"/rawData"
print('filepath selected: {}'.format(input_data_path))

filepath selected: /content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset/rawData


In [ ]:
# Global variables

distances=[i for i in range(1,26)]
nsamples=50
span_length=80
nsymbols=2048

######
min_dist=0
max_dist=3000
selCP=[1,7,10,15]
selCP_pos=[(-3,3),(1,1),(-1,-1),(1,-3)]
my_centers=[[-3,3],[-1,3],[1,3],[3,3],[-3,1],[-1,1],[1,1],[3,1],[-3,-1],[-1,-1],[1,-1],[3,-1], [-3,-3],[-1,-3],[1,-3],[3,-3]]
######

#Run main

# Main v2

In [19]:
# input_data_path = path + "/Spectrum/CNN/new_working_df.csv"
def symbol2symbol_main_v2(method, quadrants, input_data_path, distances, output_path, params = None, nsymbols = 2048, min_dist = 0, max_dist = 3000):
  
  if (params is None):
    output_features = "/{}_alpha_75_beta_25_features.csv".format(str(method))
  else:
    decimal_alpha = modf(params['alpha'])
    decimal_alpha  = round(decimal_alpha[0], 2)
    decimal_alpha = re.findall('..(.*)', str(decimal_alpha))[0]

    decimal_beta = modf(params['beta'])
    decimal_beta  = round(decimal_beta[0], 2)
    decimal_beta = re.findall('..(.*)', str(decimal_beta))[0]

    output_features = "/{}_alpha_{}_beta{}_features.csv".format(str(method), str(decimal_alpha), str(decimal_beta))
  
  # Get input data
  working_df = pd.read_csv(input_data_path)
  working_df['Distance_km'] = working_df['Distance_km'].astype(int)

  #Get train/test split
  train_idxs2, test_idxs2 = get_idx_train_test_split(working_df, distances, trainingProp = 0.8)

  # Get labels and data
  distances = [x*80 for x in range(1, 26)]
  X, Y = create_df_v2(working_df, distances, nsymbols = 2048, span_length=80,  min_dist = 0, max_dist = 3000)
  X_train2 = X.iloc[train_idxs2].reset_index(drop = "True")
  X_test2 = X.iloc[test_idxs2].reset_index(drop = "True")

  '''
  data_new = {}

  for i in range(0,25):
    data_new[str(80*(i+1))] = {"Train":X_train2.iloc[i*2:(i*2+2)].reset_index(drop = "True"),"Test":X_test2.iloc[i*25:(i*25+25)].reset_index(drop = "True")}  

  first = True
  modify_all_data_df = pd.DataFrame()
  for i in tqdm(range(25)):
      print('\n')
      print("i = ",i)
      modify_data_df = pd.DataFrame()
      for j in range(i+1,25):
          print(" j = ",j)
          source = data_new[str(distances[i]*80)]['Train']
          target = data_new[str(distances[j]*80)]['Train']
          source_test = data_new[str(distances[i]*80)]['Test']

          source_test2 = source_test.copy()

          for quadrant in quadrants:
              print("   quadrant = ", quadrant)
              if method == "GD":
                M, b = compute_parameters(quadrant, 'GD', source, target, params=params)
                new_points, indexes = modify_const_GD(quadrant, source_test, M, b)
              elif method == "Z":
                mean_source,cov_source,mean_target,cov_target = compute_parameters(quadrant, 'Z', source, target, params=None)
                new_points, indexes = modify_const_Z(quadrant,source_test,mean_source,cov_source,mean_target,cov_target)

              for k in range(len(indexes)):
                  source_test2.iloc[indexes[k][0],indexes[k][1]] = complex(new_points[k][0], new_points[k][1])

          F=[]
          for k in range(source_test2.shape[0]):
              data2=list(source_test2.iloc[k,:])
              data2=[[float(d.real), float(d.imag)] for d in data2]
              gmm = GaussianMixture(n_components=16, random_state=0, means_init=my_centers).fit(data2)
              mus=gmm.means_
              sigmas=gmm.covariances_

              features=[distances[i]*80, distances[j]*80]

              for z in selCP_pos:
                  mindist=None
                  k_inc=None
                  for w in range(16):
                      d=L2dist(mus[w],z)
                      if mindist is None or mindist>d:
                          mindist=d
                          k_inc=w

                  covmat=np.concatenate(list(sigmas[k_inc])).ravel().tolist()
                  features = [*features, *mus[k_inc], *covmat]
              F.append(features)
          
          header=['original_dist', 'target_dist']
          
          for j in selCP:
              header=[*header,*['mu_r_'+str(j),'mu_i_'+str(j),'sigma_rr_'+str(j),'sigma_ri_'+str(j),'sigma_ir_'+str(j),'sigma_ii_'+str(j)]]
   
          with open(output_path + output_features, 'a', encoding='UTF8', newline='') as f:
              writer = csv.writer(f)
              # write the header
              if first:
                writer.writerow(header)
                first = False
              # write multiple rows
              writer.writerows(F)   
    '''

# Run

In [15]:
# PLEASE SELCET THE METHOD AND QUADRANTS THAT ARE DESIRED TO BE TESTED
method = "GD" # or method = "Z"
quadrants = [1, 7, 10, 15]
distances = [x*80 for x in range(1, 26)]

# PLEASE review that the path below is correct
input_data_path = path + "/Spectrum/NN/new_data_working_df.csv"

#input_data_path=path+"/rawData"
print('filepath selected: {}'.format(input_data_path))

#Output mod files
output_path = path + '/modifiedData/'+ method

# Params for gradient descent alpha and beta
a = 0.7
b = 0.2
params = {"alpha": float(a), "beta": float(b)}

filepath selected: /content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset/Spectrum/NN/new_data_working_df.csv


In [20]:
symbol2symbol_main_v2(method, quadrants, input_data_path, distances, output_path, params = None, nsymbols = 2048, min_dist = 0, max_dist = 3000)

     -39.90625    -39.875  -39.84375   -39.8125  -39.78125     -39.75  \
0   -57.249552 -57.624664 -58.071283 -60.040833 -54.626309 -65.738997   
1   -63.880076 -54.838930 -50.731974 -52.124606 -55.442314 -59.147945   
2   -57.122957 -53.956785 -53.896963 -56.500870 -61.729861 -67.854404   
3   -70.696618 -65.303756 -60.683876 -57.875370 -54.574001 -53.748213   
4   -51.348821 -60.813456 -65.890334 -66.461066 -66.788831 -59.574001   
..         ...        ...        ...        ...        ...        ...   
495 -55.993777 -60.940842 -63.454978 -65.541745 -64.175338 -55.326790   
496 -56.829372 -55.905008 -56.196466 -58.846688 -57.930540 -57.942953   
497 -54.902319 -57.469566 -58.650167 -58.256577 -57.120871 -54.832446   
498 -55.869261 -59.209400 -59.939595 -57.487492 -56.982003 -53.641354   
499 -56.988001 -59.388396 -60.397553 -59.809899 -56.239314 -56.703642   

     -39.71875   -39.6875  -39.65625    -39.625  ...    34.6875   34.71875  \
0   -69.004208 -66.973126 -58.935310 -64.1246